In [ ]:
import os
import shutil

import cv2
import imutils

import numpy as np
from tqdm import tqdm

In [ ]:
%load_ext watermark
%watermark -v -m -p cv2,imutils

In [ ]:
def read_image(file_name: str, margin: int = 1000):
    # Open image
    img = cv2.imread(file_name)
    img = cv2.copyMakeBorder(img, margin, margin, margin, margin, borderType=cv2.BORDER_CONSTANT, value=(255, 255, 255))

    # Resize the image
    rsz = imutils.resize(img, height=1000)

    # Calculate the ratio
    ratio = img.shape[0] / rsz.shape[0]

    return img, rsz, ratio

def get_rect(img):
    # Convert to gray-scale
    gry = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Apply Gaussian-blur
    blr = cv2.GaussianBlur(gry, (5, 5), 0)
    
    # Apply threshold
    thr = cv2.adaptiveThreshold(blr, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 11, 0)

    # Find and grab contours
    cnt = cv2.findContours(thr.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnt = imutils.grab_contours(cnt)

    # Loop over contours
    rect = None
    max_prm = 0
    for c in cnt:
        prm = cv2.arcLength(c, True)  # perimeter
        apx = cv2.approxPolyDP(c, 0.09 * prm, True)  # approximation

        if len(apx) == 4 and max_prm < prm:
            max_prm = prm
            rect = apx
            
    return rect

def sort_by_x(corner_with_index):
    idx, (x, y) = corner_with_index
    return x

def sort_by_y(corner_with_index):
    idx, (x, y) = corner_with_index
    return y

def get_corners(rect, no_rotation_angle=2):
    no_rotation_ratio = np.tan(no_rotation_angle * np.pi/180)

    corners = rect[:,0,:].tolist()
    top_left, top_right, bottom_right, bottom_left = [None]*4

    # no rotation
    x, y, w, h = cv2.boundingRect(rect)
    x_delta = no_rotation_ratio * w
    y_delta = no_rotation_ratio * h
    for coord in corners:
        if   (np.abs(coord[0] -  x)    <= x_delta) and (np.abs(coord[1] -  y)    <= y_delta):
            top_left = coord
        elif (np.abs(coord[0] - (x+w)) <= x_delta) and (np.abs(coord[1] -  y)    <= y_delta):
            top_right = coord
        elif (np.abs(coord[0] - (x+w)) <= x_delta) and (np.abs(coord[1] - (y+h)) <= y_delta):
            bottom_right = coord
        elif (np.abs(coord[0] -  x)    <= x_delta) and (np.abs(coord[1] - (y+h)) <= y_delta):
            bottom_left = coord

    if top_left and top_right and bottom_right and bottom_left:
        return top_left, top_right, bottom_right, bottom_left

    corners_with_index = list(enumerate(corners))

    x_order = sorted(corners_with_index, key=sort_by_x)
    y_order = sorted(corners_with_index, key=sort_by_y)

    # counter-clockwise rotation
    if x_order[0][0] == y_order[1][0]: top_left     = x_order[0][1]
    if x_order[2][0] == y_order[0][0]: top_right    = x_order[2][1]
    if x_order[3][0] == y_order[2][0]: bottom_right = x_order[3][1]
    if x_order[1][0] == y_order[3][0]: bottom_left  = x_order[1][1]

    if top_left and top_right and bottom_right and bottom_left:
        return top_left, top_right, bottom_right, bottom_left

    # clockwise rotation
    if x_order[1][0] == y_order[0][0]: top_left     = x_order[1][1]
    if x_order[3][0] == y_order[1][0]: top_right    = x_order[3][1]
    if x_order[2][0] == y_order[3][0]: bottom_right = x_order[2][1]
    if x_order[0][0] == y_order[2][0]: bottom_left  = x_order[0][1]

    if top_left and top_right and bottom_right and bottom_left:
        return top_left, top_right, bottom_right, bottom_left

    raise ValueError('Incorrect input rectangle: {}\nTL - {}, TR - {}, BR - {}, BL - {}'\
                     .format(corners, top_left, top_right, bottom_right, bottom_left))

def get_angles(rect):
    # Calculate angles
    angles = []
    if not (rect is None):
        top_left, top_right, bottom_right, bottom_left = get_corners(rect)

        # find angle in assumption of counter-clockwise rotation
        angles.append(np.arctan( (top_left[1]-top_right[1])/(top_right[0]-top_left[0]) ) * 180 / np.pi)
        angles.append(np.arctan( (bottom_right[0]-top_right[0])/(bottom_right[1]-top_right[1]) ) * 180 / np.pi)
        angles.append(np.arctan( (bottom_left[1]-bottom_right[1])/(bottom_right[0]-top_left[0]) ) * 180 / np.pi)
        angles.append(np.arctan( (bottom_left[0]-top_left[0])/(bottom_left[1]-top_left[1]) ) * 180 / np.pi)
            
    return angles

def rotate(image, angle, center = None, scale = 1.0):
    (h, w) = image.shape[:2]

    if center is None:
        center = (w / 2, h / 2)

    # Perform the rotation
    M = cv2.getRotationMatrix2D(center, angle, scale)
    rotated = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_LINEAR, borderValue=(255, 255, 255))

    return rotated

In [ ]:
class Status():
    SHAPE_ERROR = '_shape'
    ROTATION_ERROR = '_rotation'
    SIZE_ERROR = '_size'
    
    def __init__(self):
        self.error = None
        self.msg = None
        
    def has_error(self):
        return not (self.error is None)
    
    def __str__(self):
        msg = 'Unknown error'
        if self.error == Status.SHAPE_ERROR:
            msg = 'Incorrect rectangle shape: {}'.format(self.msg)
        elif self.error == Status.ROTATION_ERROR:
            msg = 'Different rectangle shape: {}'.format(self.msg)
        elif self.error == Status.SIZE_ERROR:
            msg = 'Wrong rectangle shape: {}'.format(self.msg)

        return msg

def validate_angles(angles, status):
    if status.has_error():
        return
    angle_std = np.std(angles)
    if 2.0 < angle_std:
        status.error = Status.SHAPE_ERROR
        status.msg = 'Std of angles {} is {:.2f}'.format([round(angle, 2) for angle in angles], angle_std)

def validate_rotation(rect, rect_rotated, status):
    if status.has_error():
        return
    prm = cv2.arcLength(rect, True)
    prm_rotated = cv2.arcLength(rect_rotated, True)
    
    prm_diff = abs(prm - prm_rotated)/max(prm, prm_rotated)
    if 0.1 < prm_diff:
        status.error = Status.ROTATION_ERROR
        status.msg = 'Perimeter befor {:.2f} and after {:.2f} rotation is different by {:.2f}%'.format(prm, prm_rotated, 100*prm_diff)

def validate_size(rsz, rect, status):
    if status.has_error():
        return
    gry = cv2.cvtColor(rsz, cv2.COLOR_BGR2GRAY)
    mask_orig = dict(zip(*np.unique(cv2.threshold(gry, 250, 255, 0)[1], return_counts=True)))
    
    x, y, w, h = cv2.boundingRect(rect)
    gry[y:y+h, x:x+w] = 255
    mask_crop = dict(zip(*np.unique(cv2.threshold(gry, 250, 255, 0)[1], return_counts=True)))
    
    no_crop = mask_crop[0]/mask_orig[0]
    if 0.05 < no_crop:
        status.error = Status.SIZE_ERROR
        status.msg = 'Photo area befor {} and after {} isn\'t cropped by {:.2f}%'.format(mask_crop[255], mask_orig[255], 100*no_crop)

In [ ]:
def cv2_crop_photo(file_name, iputdir='', oputdir=''):
    status = Status()
    
    img, rsz, ratio = read_image(os.path.join(iputdir, file_name), margin=0)
    rect = get_rect(rsz)

    angles = get_angles(rect)
    validate_angles(angles, status)    
    angle = -np.median(angles)
    
    rsz_rotated = rotate(rsz, angle)
    rect_rotated = get_rect(rsz_rotated)
    validate_rotation(rect, rect_rotated, status)
    validate_size(rsz_rotated, rect_rotated, status)
    
    rect_rotated = ((rect_rotated.astype('float'))*ratio).astype('int')
    x, y, w, h = cv2.boundingRect(rect_rotated)

    img_rotated = rotate(img, angle)
    img_cropped = img_rotated[y:y+h, x:x+w, :]
    cv2.imwrite(os.path.join(oputdir, file_name), img_cropped)
    if status.has_error():
        os.rename(os.path.join(oputdir, file_name), os.path.join(oputdir, (status.error).join(os.path.splitext(file_name))))
        raise ValueError(status)

In [ ]:
iputdir = 'photo_2_png'
oputdir = iputdir.replace('_png', '_crop')
os.makedirs(oputdir, exist_ok=True)

for png_name in tqdm([png for png in os.listdir(iputdir) if png.endswith('.png')]):
    try:
        cv2_crop_photo(png_name, iputdir, oputdir)
    except Exception as ex:
        print('{}: {}'.format(png_name, ex))
        shutil.copy2(os.path.join(iputdir, png_name), os.path.join(oputdir, '_orig'.join(os.path.splitext(png_name))))
                
print("Done!")